In [1]:
import os
import math
import sys
import time
import copy
from tqdm import tqdm

import cv2

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import seaborn as sns


In [2]:
"""
- get responsible files for our target time durations (e.g., 7am to 9am)
"""
DATA_COLUMNS_NAME = ['taxiID', 'lng', 'lat', 'alt', 'time', 'azim', 'vel', 'gpstype', 'occ']

DATA_BASEDIR = "/home/gskim/data/CE545 Taxi/data"

TARGET_DATE = "2018-04-02"
TARGET_DURATION = ["070000", "100000"]

taget_date_dir = os.path.join(DATA_BASEDIR, TARGET_DATE)
taget_date_files = os.listdir(taget_date_dir)
taget_date_files.sort()

taget_date_files_in_duration = []
for filename in taget_date_files:
    if TARGET_DURATION[0] < filename[:-4] and filename[:-4] < TARGET_DURATION[1]:
        taget_date_files_in_duration.append(filename)
print(taget_date_files_in_duration)
print("\nThe number of files: ", len(taget_date_files_in_duration))

['070230.DAT', '070500.DAT', '070730.DAT', '071000.DAT', '071230.DAT', '071500.DAT', '071730.DAT', '072000.DAT', '072230.DAT', '072500.DAT', '072730.DAT', '073000.DAT', '073230.DAT', '073500.DAT', '073730.DAT', '074000.DAT', '074230.DAT', '074500.DAT', '074730.DAT', '075000.DAT', '075230.DAT', '075500.DAT', '075730.DAT', '080000.DAT', '080230.DAT', '080500.DAT', '080730.DAT', '081000.DAT', '081230.DAT', '081500.DAT', '081730.DAT', '082000.DAT', '082230.DAT', '082500.DAT', '082730.DAT', '083000.DAT', '083230.DAT', '083500.DAT', '083730.DAT', '084000.DAT', '084230.DAT', '084500.DAT', '084730.DAT', '085000.DAT', '085230.DAT', '085500.DAT', '085730.DAT', '090000.DAT', '090230.DAT', '090500.DAT', '090730.DAT', '091000.DAT', '091230.DAT', '091500.DAT', '091730.DAT', '092000.DAT', '092230.DAT', '092500.DAT', '092730.DAT', '093000.DAT', '093230.DAT', '093500.DAT', '093730.DAT', '094000.DAT', '094230.DAT', '094500.DAT', '094730.DAT', '095000.DAT', '095230.DAT', '095500.DAT', '095730.DAT']

The 

In [3]:
"""
- merge all (in that time zone) files 
- make data frame per each car 
"""

dfs = []
for filename in taget_date_files_in_duration:
    # read
    df_path = os.path.join(taget_date_dir, filename)
    df = pd.read_csv(df_path)
    df.columns = DATA_COLUMNS_NAME  

    # add
    dfs.append(df)

# merge and make all-in-one dataframe   
df_all = pd.concat(dfs)   

# for memory saving 
del dfs
df_all = df_all.drop("gpstype", axis=1)

In [7]:
del df_all_tmp

df_all_tmp = df_all.sort_values(["occ"], ascending=[False])
df_all_tmp = df_all_tmp.reset_index(drop=True)

df_all_tmp

,taxiID,lng,lat,alt,time,azim,vel,occ
0,180775240,1270248590,375307076,272,20180402083933,151,14,1
1,180722406,1270921383,375442666,463,20180402083434,297,65,1
2,180700357,1270080733,374920950,711,20180402083439,242,0,1
3,180722406,1271072950,375307100,362,20180402083224,320,107,1
4,180722406,1271052316,375327316,421,20180402083234,320,100,1
5,180722406,1271031966,375347316,461,20180402083244,320,105,1
6,180722406,1271011666,375367100,434,20180402083254,320,102,1
7,180722406,1270998566,375380433,383,20180402083304,321,71,1
8,180722406,1270981133,375394866,360,20180402083314,314,79,1
9,180722406,1270967066,375405400,315,20180402083324,314,55,1


In [4]:
# grouping by taxi ID
taxi_grouped = df_all.groupby('taxiID')
taxi_grouped.count()

,lng,lat,alt,time,azim,vel,occ
taxiID,,,,,,,
180000525,210,210,210,210,210,210,210
180000826,871,871,871,871,871,871,871
180002092,1035,1035,1035,1035,1035,1035,1035
180002578,124,124,124,124,124,124,124
180003216,1050,1050,1050,1050,1050,1050,1050
180100817,1035,1035,1035,1035,1035,1035,1035
180103394,884,884,884,884,884,884,884
180104894,1035,1035,1035,1035,1035,1035,1035
180105156,6,6,6,6,6,6,6


In [ ]:
for key in taxi_grouped.groups:
    print(key)

In [ ]:
# fig, ax = plt.subplots(figsize=(10, 10))
# ax = sns.lineplot(x="time", y="occ", data=taxi)
# plt.xlabel('passenger on and off graph ', fontsize=15)
# plt.show()


In [5]:
# print(np.count_nonzero(taxi_np_diff == EVENT_ON_TO_OFF), "passengers got off the taxi ID", taxi_id)  
#     print(np.count_nonzero(taxi_np_diff == EVENT_OFF_TO_ON), "passengers took the taxi ID", taxi_id)

EVENT_OFF_TO_ON = -1
EVENT_ON_TO_OFF = 1

taxi_id_dict = taxi_grouped.groups

for for_idx, taxi_id in enumerate(taxi_id_dict):
    taxi = taxi_grouped.get_group(taxi_id)
    taxi = taxi.sort_values(["time"], ascending=[True])
    taxi = taxi.reset_index(drop=True)

    taxi_np = taxi.values
    taxi_np = taxi_np[1:, :]
    taxi_np_occ_diff = taxi_np[1:, -1] - taxi_np[:-1, -1]
    for occ_idx, occ_flag in enumerate(taxi_np_occ_diff):
        if(occ_flag == EVENT_OFF_TO_ON):
            print("A passenger took the taxi (" + str(taxi_id) + ")", 
                  "at time", taxi_np[occ_idx, 4],
                  "and (lng, lat) =", taxi_np[occ_idx, 1], taxi_np[occ_idx, 2])
            occ_time_in = taxi_np[occ_idx, 4]

        if(occ_flag == EVENT_ON_TO_OFF):
            print("A passenger got off the taxi (" + str(taxi_id) + ")", 
                  "at time", taxi_np[occ_idx, 4],
                  "and (lng, lat) =", taxi_np[occ_idx, 1], taxi_np[occ_idx, 2])
            occ_time_out = taxi_np[occ_idx, 4]
            
            minute_diff = float(str(occ_time_out)[-4:-2]) - float(str(occ_time_in)[-4:-2])
            print(minute_diff, "minuites.\n")


A passenger took the taxi (180000525) at time 20180402072450 and (lng, lat) = 1264522355 374488455
A passenger got off the taxi (180000826) at time 20180402071148 and (lng, lat) = 1271477105 375004535
-13.0 minuites.

A passenger took the taxi (180000826) at time 20180402071648 and (lng, lat) = 1271290335 375031865
A passenger got off the taxi (180000826) at time 20180402072219 and (lng, lat) = 1271460485 374983965
6.0 minuites.

A passenger took the taxi (180000826) at time 20180402074401 and (lng, lat) = 1271732085 375510565
A passenger got off the taxi (180000826) at time 20180402075804 and (lng, lat) = 1271710735 375462165
14.0 minuites.

A passenger took the taxi (180000826) at time 20180402082157 and (lng, lat) = 1271039755 375217755
A passenger got off the taxi (180000826) at time 20180402083839 and (lng, lat) = 1271326335 375443455
17.0 minuites.

A passenger took the taxi (180000826) at time 20180402084239 and (lng, lat) = 1271389185 375339115
A passenger got off the taxi (180

A passenger got off the taxi (180213076) at time 20180402070930 and (lng, lat) = 1268495855 375679905
-11.0 minuites.

A passenger took the taxi (180213076) at time 20180402073250 and (lng, lat) = 1268716115 375250505
A passenger got off the taxi (180213076) at time 20180402073830 and (lng, lat) = 1268634435 375360705
6.0 minuites.

A passenger took the taxi (180213076) at time 20180402074710 and (lng, lat) = 1268748635 375463705
A passenger got off the taxi (180213076) at time 20180402075130 and (lng, lat) = 1268745485 375498655
4.0 minuites.

A passenger took the taxi (180213076) at time 20180402080530 and (lng, lat) = 1268872685 375822615
A passenger got off the taxi (180213076) at time 20180402081550 and (lng, lat) = 1268972505 375776405
10.0 minuites.

A passenger took the taxi (180213076) at time 20180402081840 and (lng, lat) = 1268922235 375772785
A passenger got off the taxi (180213076) at time 20180402082710 and (lng, lat) = 1268972515 375776385
9.0 minuites.

A passenger took

A passenger took the taxi (180400022) at time 20180402072310 and (lng, lat) = 1269407635 375196465
A passenger got off the taxi (180400022) at time 20180402072320 and (lng, lat) = 1269404815 375195615
0.0 minuites.

A passenger took the taxi (180400022) at time 20180402081120 and (lng, lat) = 1270447365 375245255
A passenger got off the taxi (180400022) at time 20180402081150 and (lng, lat) = 1270446285 375245465
0.0 minuites.

A passenger took the taxi (180400022) at time 20180402081310 and (lng, lat) = 1270426865 375245665
A passenger got off the taxi (180400022) at time 20180402081330 and (lng, lat) = 1270443255 375248055
0.0 minuites.

A passenger took the taxi (180400022) at time 20180402082000 and (lng, lat) = 1270434665 375270705
A passenger got off the taxi (180400022) at time 20180402082020 and (lng, lat) = 1270456435 375265285
0.0 minuites.

A passenger took the taxi (180400022) at time 20180402082550 and (lng, lat) = 1270475355 375234835
A passenger got off the taxi (1804000

A passenger took the taxi (180406933) at time 20180402094030 and (lng, lat) = 1271130885 375172505
A passenger got off the taxi (180406933) at time 20180402094040 and (lng, lat) = 1271132385 375171515
0.0 minuites.

A passenger took the taxi (180407046) at time 20180402085810 and (lng, lat) = 1270747235 376262885
A passenger got off the taxi (180407046) at time 20180402085820 and (lng, lat) = 1270742005 376261365
0.0 minuites.

A passenger took the taxi (180407046) at time 20180402091120 and (lng, lat) = 1270571015 376533515
A passenger got off the taxi (180407046) at time 20180402091130 and (lng, lat) = 1270578455 376535935
0.0 minuites.

A passenger took the taxi (180407046) at time 20180402091820 and (lng, lat) = 1270637115 376464055
A passenger got off the taxi (180407046) at time 20180402091840 and (lng, lat) = 1270644305 376470115
0.0 minuites.

A passenger took the taxi (180407046) at time 20180402094040 and (lng, lat) = 1270655785 376569435
A passenger got off the taxi (1804070

A passenger got off the taxi (180412940) at time 20180402082443 and (lng, lat) = 1271407685 376492105
-26.0 minuites.

A passenger took the taxi (180412940) at time 20180402084424 and (lng, lat) = 1270848155 376206555
A passenger got off the taxi (180412940) at time 20180402084814 and (lng, lat) = 1270794155 376236055
4.0 minuites.

A passenger took the taxi (180412940) at time 20180402085244 and (lng, lat) = 1270785365 376345055
A passenger got off the taxi (180412940) at time 20180402085954 and (lng, lat) = 1270739255 376396865
7.0 minuites.

A passenger took the taxi (180412940) at time 20180402090624 and (lng, lat) = 1270599815 376544215
A passenger got off the taxi (180412995) at time 20180402070914 and (lng, lat) = 1270985265 375025035
3.0 minuites.

A passenger took the taxi (180412995) at time 20180402072254 and (lng, lat) = 1271105365 375268885
A passenger got off the taxi (180412995) at time 20180402073634 and (lng, lat) = 1271272315 375313885
14.0 minuites.

A passenger took

...

A passenger got off the taxi (180702586) at time 20180402072444 and (lng, lat) = 1269110950 374505500
-34.0 minuites.

A passenger took the taxi (180702586) at time 20180402072954 and (lng, lat) = 1268950933 374558466
A passenger got off the taxi (180702586) at time 20180402073816 and (lng, lat) = 1269130366 374501050
9.0 minuites.

A passenger took the taxi (180702586) at time 20180402075853 and (lng, lat) = 1269610166 374779416
A passenger got off the taxi (180702586) at time 20180402075913 and (lng, lat) = 1269608183 374776249
1.0 minuites.

A passenger took the taxi (180702586) at time 20180402080005 and (lng, lat) = 1269596166 374780316
A passenger got off the taxi (180702586) at time 20180402080525 and (lng, lat) = 1269526233 374810233
5.0 minuites.

A passenger took the taxi (180702586) at time 20180402081135 and (lng, lat) = 1269510483 374623016
A passenger got off the taxi (180702586) at time 20180402081617 and (lng, lat) = 1269420900 374710083
5.0 minuites.

A passenger took 

A passenger took the taxi (180702706) at time 20180402072308 and (lng, lat) = 1270178476 375415393
A passenger got off the taxi (180702706) at time 20180402072758 and (lng, lat) = 1270194681 375469930
4.0 minuites.

A passenger took the taxi (180702706) at time 20180402073628 and (lng, lat) = 1270207985 375657620
A passenger got off the taxi (180702706) at time 20180402073718 and (lng, lat) = 1270197768 375656741
1.0 minuites.

A passenger took the taxi (180702706) at time 20180402074948 and (lng, lat) = 1270162773 375543830
A passenger got off the taxi (180702706) at time 20180402075848 and (lng, lat) = 1270241255 375623083
9.0 minuites.

A passenger took the taxi (180702706) at time 20180402081928 and (lng, lat) = 1270269548 376243590
A passenger got off the taxi (180702706) at time 20180402082506 and (lng, lat) = 1270243600 376309245
6.0 minuites.

A passenger took the taxi (180702706) at time 20180402082856 and (lng, lat) = 1270382098 376323593
A passenger got off the taxi (1807027

A passenger got off the taxi (180702842) at time 20180402082905 and (lng, lat) = 1270992750 375939255
2.0 minuites.

A passenger took the taxi (180702842) at time 20180402085735 and (lng, lat) = 1271096911 375502298
A passenger got off the taxi (180702842) at time 20180402091535 and (lng, lat) = 1270823478 375764096
-42.0 minuites.

A passenger took the taxi (180702842) at time 20180402092816 and (lng, lat) = 1270754959 376184593
A passenger got off the taxi (180702842) at time 20180402095316 and (lng, lat) = 1270924491 375990008
25.0 minuites.

A passenger got off the taxi (180702844) at time 20180402071234 and (lng, lat) = 1270370516 374951000
-16.0 minuites.

A passenger took the taxi (180702844) at time 20180402071604 and (lng, lat) = 1270259400 374974850
A passenger got off the taxi (180702844) at time 20180402072414 and (lng, lat) = 1270435600 374932316
8.0 minuites.

A passenger took the taxi (180702844) at time 20180402073854 and (lng, lat) = 1270844550 374879016
A passenger go

12.0 minuites.

A passenger took the taxi (180702973) at time 20180402082839 and (lng, lat) = 1269792560 375381480
A passenger got off the taxi (180702973) at time 20180402083249 and (lng, lat) = 1269783050 375388510
4.0 minuites.

A passenger took the taxi (180702973) at time 20180402083949 and (lng, lat) = 1269882200 375258080
A passenger got off the taxi (180702973) at time 20180402084709 and (lng, lat) = 1269905110 375375450
8.0 minuites.

A passenger took the taxi (180702973) at time 20180402085659 and (lng, lat) = 1269730250 375256830
A passenger got off the taxi (180702973) at time 20180402091809 and (lng, lat) = 1269728250 375259060
-38.0 minuites.

A passenger took the taxi (180702973) at time 20180402092309 and (lng, lat) = 1269883360 375245560
A passenger got off the taxi (180702973) at time 20180402093751 and (lng, lat) = 1269808250 375321733
14.0 minuites.

A passenger took the taxi (180702973) at time 20180402095422 and (lng, lat) = 1269861100 375604983
A passenger got of

KeyboardInterrupt: 

In [ ]:
# print(np.count_nonzero(taxi_np_diff == EVENT_ON_TO_OFF), "passengers got off the taxi ID", taxi_id)  
#     print(np.count_nonzero(taxi_np_diff == EVENT_OFF_TO_ON), "passengers took the taxi ID", taxi_id)

EVENT_OFF_TO_ON = 1
EVENT_ON_TO_OFF = -1

taxi_id_dict = taxi_grouped.groups

taxi_on_history_start_flag = 0
taxi_off_history_start_flag = 0
for for_idx, taxi_id in enumerate(tqdm(taxi_id_dict, mininterval=20)):                

    if(for_idx % 10000 == 0):
        print(taxi_on_history.shape)
#         print(taxi_off_history.shape)

    taxi = taxi_grouped.get_group(taxi_id)
    taxi = taxi.sort_values(["time"], ascending=[True])
    taxi = taxi.reset_index(drop=True)

    taxi_np = taxi.values
    taxi_np = taxi_np[1:, :]
    taxi_np_occ_diff = taxi_np[:-1, -1] - taxi_np[1:, -1] # EVENT_OFF_TO_ON = 1, EVENT_ON_TO_OFF = -1
    for occ_idx, occ_flag in enumerate(taxi_np_occ_diff.tolist()):

        if(occ_flag == EVENT_OFF_TO_ON):
            event = taxi_np[occ_idx, :]
            event = np.expand_dims(event, axis=0)
            if(taxi_on_history_start_flag == 0):
                taxi_on_history = event
                taxi_on_history_start_flag = 1
            else:
                taxi_on_history = np.concatenate((taxi_on_history, event), axis=0)
                                
        if(occ_flag == EVENT_ON_TO_OFF):
            event = taxi_np[occ_idx, :]
            event = np.expand_dims(event, axis=0)
            if(taxi_off_history_start_flag == 0):
                taxi_off_history = event
                taxi_off_history_start_flag = 1
            else:
                taxi_off_history = np.concatenate((taxi_off_history, event), axis=0)
                


In [ ]:
taxi_on_history_ = taxi_on_history[:, :-1]
taxi_off_history_ = taxi_off_history[:, :-1]

print(taxi_on_history.shape)
print(taxi_off_history.shape)

np.savetxt("taxi_on_20180402_070000_100000.csv", taxi_on_history_, delimiter=",")
np.savetxt("taxi_off_20180402_070000_100000.csv", taxi_off_history_, delimiter=",")
